In [9]:
import os
import sys
from dotenv import load_dotenv # type: ignore
#import pandas as pd

env_path = '../.env'
load_dotenv(dotenv_path=env_path)
path = os.getenv('HOME_PATH')
sys.path.append(path)

from Class.QueryBuilder import QueryBuilder

In [10]:
qb = QueryBuilder()
conn = qb.get_connection(path + "data/interim/churn.db")
query = qb.select("gender, customer_id").from_table("churn").limit(5).build()

result = conn.execute(query).df()
result

,gender,customer_id
0,Female,15634602
1,Female,15647311
2,Female,15619304
3,Female,15701354
4,Female,15737888


In [11]:
query = qb.column_names("churn").build()
result = conn.execute(query).fetchdf()
result


,column_name
0,row_number
1,customer_id
2,surname
3,credit_score
4,geography
5,gender
6,age
7,tenure
8,balance
9,num_of_products
